#Libraries

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Dataset

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/BEPE/datasets/all_replies_goEmotions.csv')

In [ ]:
df.drop(columns=['Unnamed: 0'], inplace=True)
df.head()

,tweet_id,created_at,timestamp_ms,tweet_text,id_tweet_reply,predictions,scores
0,1246155262374957058,Fri Apr 03 19:19:01 +0000 2020,1585941541652,"No vídeo, 6:30 - 6:40, fala que vai sancionar...",1244963529377153024,neutro,0.780009
1,1246165338254979077,Fri Apr 03 19:59:03 +0000 2020,1585943943929,Já viu isso Sr° Presidente?,1244963529377153024,curiosidade,0.754949
2,1246257392150552580,Sat Apr 04 02:04:51 +0000 2020,1585965891288,A população que nunca sai daí né? Mesmo reban...,1244963529377153024,raiva,0.304363
3,1245329530476257280,Wed Apr 01 12:37:51 +0000 2020,1585744671826,Você é louco. Se não fosse a atitude firme do...,1244963529377153024,raiva,0.488081
4,1245331502214283264,Wed Apr 01 12:45:41 +0000 2020,1585745141925,Quem te segue é mais bandido que tu .,1244963529377153024,neutro,0.297641


# Inference

In [ ]:
infer = False

In [ ]:
if infer:
  # Folder path containing the fine-tuned model files
  model_path = '/content/drive/MyDrive/IC/Inferencia_carga_afetiva/GoEmotions_portuguese-main/fine_tunel_model'
  model = BertForSequenceClassification.from_pretrained(model_path)
  tokenizer = BertTokenizer.from_pretrained(model_path)
  classifier = pipeline('text-classification', model=model, tokenizer=tokenizer, top_k=1, device=0)

In [ ]:
def inference(tweets):
  return classifier(tweets)

In [ ]:
# s = start
# e = end
def save_inference(s, e):
  tweets = list(df['tweet_text'])
  pred = list(df['predictions'][:s])
  score = list(df['scores'][:s])
  emotions = inference(tweets[s:e])
  for emotion in emotions:
    pred.append(emotion[0]['label'])
    score.append(emotion[0]['score'])
  for i in range(len(df) - e):
    pred.append('-')
    score.append('-')
  df['predictions'] = pred
  df['scores'] = score
  df.to_csv('/content/drive/MyDrive/BEPE/datasets/all_replies_goEmotions.csv')

In [ ]:
save_inference(4500000, 4971545)